In [13]:
import os
print(os.getcwd())


c:\Users\Mrunmayee\OneDrive\Desktop\quick-commerce-ai\notebooks


In [1]:
import pandas as pd
import numpy as np
import holidays
from sklearn.preprocessing import MinMaxScaler
import os


In [2]:

# Ensure correct working directory
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(project_root)

print("Working directory:", os.getcwd())

Working directory: c:\Users\Mrunmayee\OneDrive\Desktop\quick-commerce-ai


In [3]:
# Load data
sales = pd.read_csv("data/sales_train_evaluation.csv")
calendar = pd.read_csv("data/calendar.csv")


In [4]:
# Keep only mapping fields
calendar = calendar[["d", "date"]]
calendar["date"] = pd.to_datetime(calendar["date"])

In [5]:
# Melt M5 sales to long format
sales = sales.melt(id_vars=["item_id"], var_name="d", value_name="sales")

# Merge real dates
sales = sales.merge(calendar, on="d", how="left")
sales.drop(columns=["d"], inplace=True)

In [ ]:
# Sort by date
sales = sales.sort_values(by="date").reset_index(drop=True)

# Fill missing
sales["sales"] = sales["sales"].fillna(0)

# Aggregate ONLY by hour (across all SKUs)
sales = sales.groupby(pd.Grouper(key="date", freq="1H")).sum().reset_index()

# Feature Engineering
sales["hour"] = sales["date"].dt.hour
sales["day"] = sales["date"].dt.dayofweek
sales["is_weekend"] = sales["day"].isin([5, 6]).astype(int)


C:\Users\Mrunmayee\AppData\Local\Temp\ipykernel_17232\1960596847.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  sales = sales.groupby(pd.Grouper(key="date", freq="1H")).sum().reset_index()


In [ ]:
# Add holiday feature
ind_holidays = holidays.IN()
sales["is_holiday"] = sales["date"].dt.date.isin(ind_holidays).astype(int)


In [ ]:

# Scale data for ML
scaler = MinMaxScaler()
sales["sales_scaled"] = scaler.fit_transform(sales[["sales"]])

print("Data prepared successfully!")
print(sales.head(), sales.shape)


Working directory: c:\Users\Mrunmayee\OneDrive\Desktop\quick-commerce-ai


C:\Users\Mrunmayee\AppData\Local\Temp\ipykernel_29052\1745389687.py:35: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  sales = sales.groupby(pd.Grouper(key="date", freq="1H")).sum().reset_index()


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

SEQ_LEN = 48  # 48 hours history
PRED_LEN = 4  # 4 hours forecast

class DemandDataset(Dataset):
    def __init__(self, df):
        self.data = df["sales_scaled"].values

    def __len__(self):
        return len(self.data) - SEQ_LEN - PRED_LEN

    def __getitem__(self, idx):
        x = self.data[idx:idx+SEQ_LEN]
        y = self.data[idx+SEQ_LEN:idx+SEQ_LEN+PRED_LEN]
        return torch.tensor(x).float(), torch.tensor(y).float()

dataset = DemandDataset(sales)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

class LSTMForecast(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(1, 64, batch_first=True)
        self.fc = nn.Linear(64, PRED_LEN)

    def forward(self, x):
        x = x.unsqueeze(-1)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

model = LSTMForecast()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    for x, y in loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")


NameError: name 'sales' is not defined

In [ ]:
torch.save(model.state_dict(), "models/demand_forecast.pt")
